### AutoClass

*Este tutorial es una adaptación de:* https://huggingface.co/docs/transformers/v4.47.1/es/autoclass_tutorial

Es útil tener la posibilidad de cargar un modelo de arquitectura transformers preentrenado desde un *checkpoint* determinado. Los *checkpoints* son puntos  de entrenamiento del modelo para los que se han almacenado todos los elementos que constituyen el estado del modelo, de forma que este estado pueda ser fielmente replicado.

Por ejemplo, para la arquitectura de modelo Bert, existen varios checkpoints como por ejemplo: bert-base-uncased, bert-base-cased, bert-base-multlingual-case, etc... Cada uno de estos checkpoints contiene los valores de los parámetros del modelo entrenados para operar con ciertos condicionantes o restricciones utilizando el modelo Bert. Por ejemplo, la diferencia entre las versiones cased and uncased es si hacen distinción o no entre mayúsculas y minúsculas. Las versiones etiquetadas como multilingual hacen referencia a la capacidad del modelo para operar en diferentes idiomas. Normalmente los modelos están entrenados para un idioma base, que suele ser el inglés para el mundo occidental, pero en China el idioma base suele ser el chino.

La familia de las *AutoClass* son un bloque constructor básico de la librería Transformer. Permiten la utilización de Modelos y de distintos componentes de los mismos, sin tener que elegir la clase adecuada para el checkpoint que se está utilizando. La AutoClass correspondiente se encargará de cargar el componente adecuado para la arquitectura del modelo cargado.

Existen *AutoClass* tanto para modelos como para distintos componentes de los mismos:

- Tokenizer: AutoTokenizer
- ImageProcessor: AutoImageProcessor
- Backbone (Embedding): AutoBackbone
- Feature Extractor: AutoFeatureExtractor
- Processor: AutoProcessor
- Modelo: AutoModel

Veamos ahora ejemplos de varios de estos componentes, empezando por el **AutoTokenizer**

In [1]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
sequence = "In a hole in the ground there lived a hobbit."
print(tokenizer(sequence))

{'input_ids': [101, 1999, 1037, 4920, 1999, 1996, 2598, 2045, 2973, 1037, 7570, 10322, 4183, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


## Tareas de procesamiento de imágenes

Los *tokenizers* son útiles para tareas de lenguaje, en el caso de modelos cuya entrada es una imagen, se necesita un *Image Processor* que transforme la imagen al formato correcto.

In [4]:
from transformers import AutoImageProcessor, AutoBackbone
import torch
from PIL import Image
import requests
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
processor = AutoImageProcessor.from_pretrained("microsoft/swin-tiny-patch4-window7-224")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


Siguiendo el flujo de procesamiento de una imagen de entrada, la clase *AutoBackbone* permite utilizar modelos como backbones para extaer los mapas de características de diferentes etapas (o capas) del backbone. En este caso, además de cargar un checkpoint determinado utilizando la función *from_pretrained* puedes indicar como parámetro las capas de las que quieres obtener el mapa de características usando una de estas dos formas:
- **out_indices**: es el índice de las capas de las que querrías obtener el mapa de características
- **out_features**: es el nombre de las capas de las que querrías obtener el mapa de características

Veamos un ejemplo, siguiendo el código anterior.

In [6]:
model = AutoBackbone.from_pretrained("microsoft/swin-tiny-patch4-window7-224", out_indices=(1,))
inputs = processor(image, return_tensors="pt")
outputs = model(**inputs)
feature_maps = outputs.feature_maps
list(feature_maps[0].shape)

Some weights of the model checkpoint at microsoft/swin-tiny-patch4-window7-224 were not used when initializing SwinBackbone: ['classifier.bias', 'classifier.weight', 'swin.layernorm.bias', 'swin.layernorm.weight']
- This IS expected if you are initializing SwinBackbone from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SwinBackbone from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of SwinBackbone were not initialized from the model checkpoint at microsoft/swin-tiny-patch4-window7-224 and are newly initialized: ['swin.hidden_states_norms.stage1.weight', 'swin.hidden_states_norms.stage1.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and

[1, 96, 56, 56]

## Tareas de procesamiento de audio

Para tareas de procesamiento de audio podemos usar la clase AutoFeatureExractor para procesar la señal de audio al formato de entrada adecuado.

In [7]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained(
    "ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition"
)

## Tareas multimodales

Las tareas multimodales requieren un procesado que combine dos tipo de herramientas de preprocesado. Por ejemplo, el modelo LayoutLMV2 necesita un procesador de imágenes para manejar imágenes y un tokenizer para manejar texto. Usando un AutoProcessor podemos generar un preprocesador que combine ambas.

In [8]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained("microsoft/layoutlmv2-base-uncased")

# Modelos

La familia de clases AutoModelFor permite la carga de un modelo preentrenado para una tare determinada. En [este enlace](https://huggingface.co/docs/transformers/model_doc/auto#generic-pretraining-classes) podemos encontrar la lista completa. Veamos un ejemplo con la clase *AutoModelForSequenceClassification*.


In [9]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased", torch_dtype="auto")

Some weights of the model checkpoint at distilbert/distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bia

Podemos usar el mismo checkpoint para cargar un modelo para una tarea diferente.

In [10]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained("distilbert/distilbert-base-uncased", torch_dtype="auto")

Some weights of the model checkpoint at distilbert/distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this 